# Lecture 15: Efficient Vision Models

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gaurav-redhat/transformer_problems/blob/efficientml-course/efficientml_course/15_efficient_vision_models/demo.ipynb)

MobileNet, depthwise separable convolutions, and EfficientNet.


In [ ]:
!pip install torch -q
import torch
import torch.nn as nn

# Depthwise Separable Convolution - The key to efficient CNNs
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_ch, out_ch, kernel=3):
        super().__init__()
        # Depthwise: spatial filtering (cheap)
        self.depthwise = nn.Conv2d(in_ch, in_ch, kernel, padding=kernel//2, groups=in_ch)
        # Pointwise: channel mixing (1x1)
        self.pointwise = nn.Conv2d(in_ch, out_ch, 1)
    
    def forward(self, x):
        return self.pointwise(self.depthwise(x))

# Compare FLOPs
def count_conv_flops(in_ch, out_ch, kernel, h, w):
    return 2 * in_ch * out_ch * kernel * kernel * h * w

def count_dw_sep_flops(in_ch, out_ch, kernel, h, w):
    dw = 2 * in_ch * kernel * kernel * h * w
    pw = 2 * in_ch * out_ch * h * w
    return dw + pw

in_ch, out_ch, k, h, w = 64, 64, 3, 56, 56

std_flops = count_conv_flops(in_ch, out_ch, k, h, w)
dw_flops = count_dw_sep_flops(in_ch, out_ch, k, h, w)

print("Standard Conv vs Depthwise Separable")
print(f"  Standard Conv3x3: {std_flops/1e6:.1f} MFLOPs")
print(f"  Depthwise Sep:    {dw_flops/1e6:.1f} MFLOPs")
print(f"  Speedup: {std_flops/dw_flops:.1f}x fewer FLOPs!")
print("\n🎯 This is why MobileNet is so fast!")
